## Install dependency

In [1]:
! pip install cryptography

## How it works (high level)

Master password → key (via PBKDF2)

Passwords stored in an encrypted JSON file

File is unreadable without the master password

In [2]:
import os
import json
import base64
from getpass import getpass
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet

DATA_FILE = "vault.enc"
SALT_FILE = "salt.bin"


def generate_key(master_password: str) -> Fernet:
    if not os.path.exists(SALT_FILE):
        salt = os.urandom(16)
        with open(SALT_FILE, "wb") as f:
            f.write(salt)
    else:
        with open(SALT_FILE, "rb") as f:
            salt = f.read()

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=390000,
    )

    key = base64.urlsafe_b64encode(
        kdf.derive(master_password.encode())
    )
    return Fernet(key)


def load_vault(fernet: Fernet):
    if not os.path.exists(DATA_FILE):
        return {}

    with open(DATA_FILE, "rb") as f:
        encrypted_data = f.read()

    decrypted_data = fernet.decrypt(encrypted_data)
    return json.loads(decrypted_data.decode())


def save_vault(vault: dict, fernet: Fernet):
    encrypted_data = fernet.encrypt(json.dumps(vault).encode())
    with open(DATA_FILE, "wb") as f:
        f.write(encrypted_data)


def add_password(vault: dict):
    site = input("Site name: ")
    username = input("Username: ")
    password = getpass("Password: ")

    vault[site] = {
        "username": username,
        "password": password
    }
    print("Password saved.")


def get_password(vault: dict):
    site = input("Site name: ")
    if site in vault:
        print(f"Username: {vault[site]['username']}")
        print(f"Password: {vault[site]['password']}")
    else:
        print("No entry found.")


def main():
    master_password = getpass("Master password: ")
    fernet = generate_key(master_password)

    try:
        vault = load_vault(fernet)
    except Exception:
        print("Invalid master password or corrupted vault.")
        return

    while True:
        print("\n1. Add password")
        print("2. Get password")
        print("3. Exit")

        choice = input("> ")

        if choice == "1":
            add_password(vault)
            save_vault(vault, fernet)
        elif choice == "2":
            get_password(vault)
        elif choice == "3":
            break
        else:
            print("Invalid choice")


if __name__ == "__main__":
    main()



1. Add password
2. Get password
3. Exit
Password saved.

1. Add password
2. Get password
3. Exit
